In [10]:
library(rdydisstools)
setwd('~/notebooks/dissertation')
loadpackages(c('doParallel', 'foreach', 'logging', 'glue', 'grid', 'gridExtra', 'dplyr', 'ggplot2', 'DescTools', 'careless'))
nDatasets <- 5

In [14]:
registerDoParallel(72)
h1data <- foreach(i=1:nDatasets, .verbose=FALSE) %dopar% {
    # Initialize logging
    addHandler(writeToFile, file=glue::glue('~/notebooks/dissertation/logs/h1/sim{i}.log'), level='INFO')
    logstring <- glue::glue('dis.h1.{i}')
    
    # Specify path for sample
    datasetstring <- glue::glue("~/notebooks/dissertation/data/simsample{i}.RData")
    logging::loginfo(glue::glue('dataset loading from data/simsample{i}'), logger=logstring)
    
    #load sample dataset into memory
    load(datasetstring)
    logging::loginfo(glue::glue('dataframe has been loaded from: {datasetstring}'), logger=logstring)
    
    #Evaluate Hypothesis 1
    t(dispatchHypothesisEval(df, hypothesis="h1", lastColumn=100))
} %>% do.call("rbind.data.frame", .)

save(h1data, file='~/notebooks/dissertation/artifacts/h1/h1data.RData')

In [15]:
head(h1data)

hc_b,hc_lower,hc_upper,ht_b,ht_lower,ht_upper,hypTest,moreInf,hasOverlap,overlap
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.09645013,0.01717263,0.1757276,0.1844889,0.1118356,0.2571422,0,1,1,0.06389201
0.17824968,0.10614927,0.2503501,0.2954545,0.2326222,0.3582869,0,1,1,0.01772792
0.28842105,0.22598709,0.3508550,0.4089417,0.3562303,0.4616531,1,1,0,0.00000000
0.23199023,0.16460503,0.2993754,0.2809009,0.2169688,0.3448330,0,1,1,0.08240660
0.38832565,0.33465732,0.4419940,0.4094455,0.3571031,0.4617879,0,1,1,0.08489089
